In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats

In [88]:
def load_data():
    mean = 5
    std_dev = 1
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 100)
    x_train, x_test = train_test_split(x, train_size=0.8)
    y_train = (x_train>6).astype(int)
    y_test = (x_test>6).astype(int)

    x_train = np.reshape(x_train, (x_train.shape[0], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], 1))

    train_data = {'x_train':x_train, 'y_train':y_train}
    test_data = {'x_test':x_test, 'y_test':y_test}
    return train_data, test_data

In [82]:
def initialize_params():
    w = np.random.rand(1,1) * 0.1
    b = np.zeros((1,1))
    params = {'w':w,'b':b}
    return params

In [83]:
def sigmoid(z):
    a = 1/(1 + np.exp(-z))
    return a

In [102]:
def forward_prop(x, params):
    w, b = params['w'], params['b']
    z = x @ w + b
    yhat = sigmoid(z)
    return yhat

In [131]:
def calculate_cost(yhat, y):
    m = y.shape[0]
    losses = (y * np.log(yhat)) + ((1 - y) * np.log(1 - yhat))
    cost = -np.sum(losses, axis=0, keepdims=True)/m
    return cost

In [132]:
train_data, test_data = load_data()

In [133]:
params = initialize_params()

In [134]:
yhat = forward_prop(train_data['x_train'], params)

In [135]:
yhat.shape

(80, 1)

In [136]:
y = np.array([[0],[0],[1]])

In [137]:
yhat = np.array([[0.1],[0.25],[0.9]])

In [138]:
cost = calculate_cost(yhat, y)

In [139]:
cost

array([[0.16613437]])